### 사용자 정의 함수

 - - -
- 함수이름: training
- 함수기능: 모델 학습 진행 함수
- 매개변수: 함수 구동 시 필요한 재료; 
	* 모델 인스턴스
	* 학습데이터셋: 피쳐 & 타겟 (학습전 Tensor화)
	* 손실함수 인스턴스
	* 최적화 인스턴스
	* 학습횟수: EPOCH
	* 배치크기: BATCH SIZE
	* 배치개수: BATCH CNT or iterator
	* 검증용 데이터셋: 피쳐 & 타겟 (학습전 Tensor화)
- 함수결과: 에포크 당 손실값 & 성능지표값 & 검증 손실값 & 성능지표값   
	* =>=> list, dict
- - -

 - - -
- 함수이름: epochTraining
- 함수기능: 에포크 단위 모델 학습 진행 함수
- 매개변수: 함수 구동 시 필요한 재료; 
	* 모델 인스턴스
	* 학습데이터셋: 피쳐 & 타겟 (학습전 Tensor화)
	* 손실함수 인스턴스
	* 최적화 인스턴스
	* 배치크기: BATCH SIZE
	* 배치개수: BATCH CNT or iterator
	* 검증용 데이터셋: 피쳐 & 타겟 (학습전 Tensor화)
- 함수결과: 손실값 & 성능지표값
	* => tuple
- - -

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optimizer
from torchmetrics.regression import R2Score
from torchmetrics.classification import F1Score


In [ ]:
# 텐서 저장 및 실행 위치 설정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
## 1번 에포크 당 손실값 & 성능지표값 반환 함수

def Testing(model, feature, target, lossFunc, scoreFunc):

	### 최적화 기능 비활성화 후 데이터셋 추출 후 학습 진행

	with torch.no_grad():

		# 텐서화---------------------------------------------------------
		# json,csv => DF 로드 가정
		# .values  => array 반환
		featureTS = torch.FloatTensor(feature.values).to(DEVICE)	
		targetTS = torch.FloatTensor(target.values).to(DEVICE)
		
		# 학습-----------------------------------------------------------
		pre_y = model(featureTS).to(DEVICE)

		# 손실 계산------------------------------------------------------
		loss = lossFunc(pre_y, targetTS)

		# 모델 점수 계산 -------------------------------------------------
		score = scoreFunc(pre_y, targetTS)

	return loss, score			# loss, r2

In [3]:
## 1번 에포크 당 손실값 & 성능지표값 반환 함수

def epochTraining(model, feature, target, lossFunc, optim, batchCnt, 
									batchSize=32, is_class=True):

	# 
	loss_total, score_total = 0, 0

	# 에포크에서 배치크기만큼 데이터셋 추출 후 학습 진행
	for batch in range(batchCnt):

		# 배치크기만큼 데이터셋 추출 및 텐서화------------------------------
		start = batch * batchSize	# 0, 32, 64, ...
		end = start + batchSize		# 32, 64, 96, ...

		# 텐서화---------------------------------------------------------
		# json,csv => DF 로드 가정
		# .values  => array 반환
		X_train = torch.FloatTensor(feature[start:end].values).to(DEVICE)	
		y_train = torch.FloatTensor(target[start:end].values).to(DEVICE)
		
		# 학습-----------------------------------------------------------
		pre_y = model(X_train)

		# 손실 계산------------------------------------------------------
		loss = lossFunc(pre_y, y_train)
		loss_total += loss.item()			# loss는 텐서라 값만 저장하기 위해

		# 모델 점수 계산 (학습용)-------------------------------------------------
		score = F1Score()(pre_y, y_train) if is_class else R2Score()(pre_y, y_train)
		score_total += score

		# 최적화: W,b 업데이트-------------------------------------------
		optim.zero_grad()
		loss.backward()

		optim.step()			# optimizer가 파라메터 가지고 있으므로
								# 업데이트는 optimizer가!

	# 에포크당 손실값 & 성능지표
	# 테스트 및 검증 함수 사용!

	return loss_total/batchCnt, 0			# loss, r2

In [4]:
EPOCHS = 10

# 모든 epoch에 대한 train, valid loss, score 저장
TV_LOSS = {'Train': [], 'VAL':[]}
TV_SCORE = {'Train':[], 'VAL':[]}

# 전체 EPOCH 학습 진행
for epoch in range(EPOCHS):
	
	# EPOCH 1번 학습 진행
	train_loss, train_score = epochTraining()

	# EPOCH 1번 검증 진행
	val_loss, val_score = testing()

	# 에포크당 학습 및 검증 결과 저장
	TV_LOSS['Train'].append(train_loss)
	TV_SCORE['Train'].append(train_score)

	TV_LOSS['VAL'].append(val_loss)
	TV_SCORE['VAL'].append(val_score)

	# 에포크당 학습 및 검증 결과 출력
	print(f'[{epoch}.{EPOCHS}]\n [Train] LOSS:  {train_loss} SCORE: {train_score}')
	print(f'[VAL] LOSS:  {val_loss} SCORE: {val_score}')

TypeError: epochTraining() missing 6 required positional arguments: 'model', 'feature', 'target', 'lossFunc', 'optim', and 'batchCnt'